# 1. Azure ML 前置准备

本章节涵盖 Azure ML 工作区的创建与连接、本地环境配置等前置准备工作。

- **1.1** 安装本地依赖
- **1.2** 创建 Azure ML 工作区(如已有工作区可跳过)
- **1.3** 连接 Azure ML 工作区

# 端到端：Azure VM 准备 + Azure ML 上使用 LoRA 微调 Qwen3-VL-4B

## 1.1 安装本地依赖(仅 Notebook 运行环境)

- 本节为你的本机/Notebook 内核安装最小依赖,便于连接 Azure ML、做数据预处理与可视化。
- 训练将在 Azure ML 计算上执行,本机不安装 torch/transformers 等大依赖。
- 若你在中国主权云(设置 AZURE_CLOUD_NAME=AzureChinaCloud),安装命令会自动切换镜像源。
- 如已有满足条件的环境,可跳过本节。

In [1]:
%%writefile requirements-notebook.txt
# 最小依赖（Notebook 端）
# 训练发生在 Azure ML 计算上，本地无需安装 torch/transformers 等大依赖
azure-ai-ml>=1.15.0
azure-identity>=1.16.0
pandas>=2.1.0
pyarrow>=15.0.0
pillow>=10.2.0
tqdm>=4.66.0
ipywidgets>=8.1.7

Writing requirements-notebook.txt


## 1.3 连接 Azure ML 工作区

完成工作区创建(或使用已有工作区)后,在本地 Notebook 环境中通过 Python SDK 连接到 Azure ML 工作区。

以下示例代码展示如何使用 `InteractiveBrowserCredential` 进行身份验证并创建 `MLClient` 对象:

## 1.2 创建 Azure Machine Learning 工作区

本节将指导你在 Azure 门户上创建 Azure Machine Learning 工作区,这是运行 Azure ML 训练作业的前置条件。

**如果你已有工作区,可跳过本节,直接进入 1.3 连接工作区。**

### 1.2.1 前置条件
- Azure 订阅具备创建 Azure ML 工作区的权限。
- 本地已安装 Azure CLI 或能够登录 Azure 门户。
- 已规划好资源组名称和工作区所在区域。

### 1.2.2 登录 Azure 门户

根据你的云环境选择对应的门户:
- **公有云**: [https://portal.azure.com/](https://portal.azure.com/)
- **中国区**: [https://portal.azure.cn/](https://portal.azure.cn/)

使用你的 Azure 账号登录。

### 1.2.3 创建 Azure Machine Learning 工作区

#### 进入创建向导
1. 在 Azure 门户主页,点击左上角的 **"创建资源"** 按钮。
   - ![创建资源入口](images/azure_ml_create_resource.png)
2. 在搜索框中输入 **"Machine Learning"**,选择 **"Azure Machine Learning"**。
   - ![搜索 ML 服务](images/azure_ml_search.png)
3. 点击 **"创建"** 按钮,进入工作区创建向导。
   - ![开始创建](images/azure_ml_create_start.png)

#### 基本信息配置
在 **"基本信息"** 页签填写以下内容:

| 字段 | 说明 | 示例值 |
|------|------|--------|
| **订阅** | 选择你的 Azure 订阅 | `Pay-As-You-Go` |
| **资源组** | 选择现有资源组或创建新资源组 | `OctWorkRG` |
| **工作区名称** | 工作区的唯一名称(3-33字符,字母数字和连字符) | `ml-cn3-01` |
| **区域** | 选择工作区部署的 Azure 区域 | `chinanorth3` 或 `eastus` |
| **存储账户** | 默认新建,或选择现有存储账户 | 默认自动创建 |
| **Key Vault** | 默认新建,用于存储密钥和凭据 | 默认自动创建 |
| **Application Insights** | 默认新建,用于监控和日志 | 默认自动创建 |
| **容器注册表** | 可选,用于存储 Docker 镜像 | 首次可留空 |

![基本信息配置](images/azure_ml_basics.png)

**重要提示**:
- **区域选择**: 确保选择的区域支持你后续需要的 GPU VM SKU(如 NVads A10 v5)。
- **资源组**: 建议将相关资源放在同一资源组,便于管理和成本跟踪。

#### 网络配置(可选)
在 **"网络"** 页签可以配置工作区的网络访问方式:
- **公共终结点(所有网络)**: 默认选项,允许从任何位置访问。
- **专用终结点**: 通过虚拟网络私密访问,适合企业安全要求。

对于开发和学习场景,使用默认的 **"公共终结点(所有网络)"** 即可。

![网络配置](images/azure_ml_network.png)

#### 高级配置(可选)
在 **"高级"** 页签可以配置:
- **数据加密**: 选择使用 Microsoft 托管密钥还是客户托管密钥。
- **标识**: 配置系统分配或用户分配的托管标识。
- **标签**: 为资源添加标签,便于分类和计费跟踪。

对于初次使用,使用默认配置即可。

![高级配置](images/azure_ml_advanced.png)

#### 检查并创建
1. 点击 **"检查 + 创建"** 按钮。
   - ![检查配置](images/azure_ml_review.png)
2. Azure 会验证你的配置,确认所有字段都正确填写且符合规则。
3. 检查配置摘要,确认无误后点击 **"创建"** 按钮。
   - ![确认创建](images/azure_ml_create_confirm.png)
4. 部署过程通常需要 **2-5 分钟**,可以在通知面板查看进度。
   - ![部署进行中](images/azure_ml_deployment.png)

### 1.2.4 验证工作区创建成功

#### 查看部署结果
1. 部署完成后,点击 **"转到资源"** 按钮。
   - ![部署完成](images/azure_ml_deployment_complete.png)
2. 进入工作区概览页面,确认以下信息:
   - 工作区名称、资源组、订阅 ID
   - 工作区 URL(用于访问 Azure ML Studio)
   - 关联的存储账户、Key Vault 等资源

![工作区概览](images/azure_ml_workspace_overview.png)

#### 访问 Azure ML Studio
1. 在工作区概览页点击 **"启动工作室"** 按钮,或直接访问:
   - **公有云**: [https://ml.azure.com/](https://ml.azure.com/)
   - **中国区**: [https://ml.azure.cn/](https://ml.azure.cn/)
2. 选择你创建的工作区,进入 Azure ML Studio 管理界面。
   - ![ML Studio 首页](images/azure_ml_studio.png)

### 1.2.5 创建计算集群(可选,也可通过代码创建)

如果你想通过门户创建计算集群,可以按以下步骤操作:

#### 进入计算页面
1. 在 Azure ML Studio 左侧导航栏,点击 **"计算"**。
2. 选择 **"计算群集"** 选项卡。
3. 点击 **"+ 新建"** 按钮。
   - ![创建计算集群](images/azure_ml_aks_create.png)

#### 配置计算集群
填写以下配置:

| 字段 | 说明 | 示例值 |
|------|------|--------|
| **计算名称** | 集群的唯一名称 | `gpu-cluster` |
| **虚拟机大小** | 选择 VM SKU(支持 GPU 的节点) | `Standard_NVads_A10_v5` |
| **最小节点数** | 最小节点数量(0表示空闲时自动缩减) | `0` |
| **最大节点数** | 最大节点数量 | `4` |
| **空闲时间** | 节点空闲多久后自动缩减 | `120` 秒 |

![配置计算集群](images/azure_ml_aks_config.png)

**重要提示**:
- **成本控制**: 设置最小节点数为0,空闲时自动缩减,避免不必要的费用。
- **GPU 支持**: 确保选择的 VM SKU 支持 GPU(NC、NV 系列)。

#### 完成创建
1. 点击 **"创建"** 按钮。
2. 计算集群创建通常需要 **5-10 分钟**,完成后状态变为 **"成功"**。

![计算集群列表](images/azure_ml_aks_list.png)

### 1.2.6 获取工作区连接信息

完成工作区创建后,你需要以下信息来通过 Python SDK 连接工作区:

#### 通过门户获取
在工作区概览页面可以找到:
- **订阅 ID**: 在 "基本信息" 部分
- **资源组**: 在 "基本信息" 部分
- **工作区名称**: 页面顶部标题

![获取连接信息](images/azure_ml_connection_info.png)

#### 通过 Azure CLI 获取
```bash
# 列出订阅
az account list --output table

# 列出资源组
az group list --output table

# 列出工作区
az ml workspace list --resource-group <资源组名称> --output table

# 显示工作区详情
az ml workspace show --name <工作区名称> --resource-group <资源组名称>
```

记录这些信息,将在下一节 **1.3 连接工作区** 中使用。

# 2. 正式操作

前置准备完成后,本章节将完成以下操作:
- 通过 Azure SDK 交互式登录并连接工作区
- 准备数据集并注册到 Azure ML
- 在 Azure ML 计算集群上运行 LLaMA-Factory 训练作业
- 导出合并模型并注册为模型资产

## 2.1 交互式登录并连接工作区

使用 Azure SDK 进行交互式身份验证,并连接到你在第 1 章创建或准备好的 Azure ML 工作区。

In [2]:
from azure.identity import InteractiveBrowserCredential, AzureAuthorityHosts
from azure.ai.ml import MLClient
import os

# 基础配置（可使用环境变量覆盖）
# SUBSCRIPTION_ID = os.getenv("AZURE_SUBSCRIPTION_ID", "3e859a28-17f7-420e-bc02-624301a676f7")
# RESOURCE_GROUP = os.getenv("AZURE_RESOURCE_GROUP", "OctWorkRG")
# WORKSPACE_NAME = os.getenv("AZUREML_WORKSPACE_NAME", "ml-cn3-01")
SUBSCRIPTION_ID = os.getenv("AZURE_SUBSCRIPTION_ID", "7a03e9b8-18d6-48e7-b186-0ec68da9e86f")
RESOURCE_GROUP = os.getenv("AZURE_RESOURCE_GROUP", "aml-rg")
WORKSPACE_NAME = os.getenv("AZUREML_WORKSPACE_NAME", "aml-hu-east-west-us2")

# 切换公有云/主权云（AzureChinaCloud）
USE_CHINA_CLOUD = False
authority_host = AzureAuthorityHosts.AZURE_CHINA if USE_CHINA_CLOUD else AzureAuthorityHosts.AZURE_PUBLIC_CLOUD
ml_client_kwargs = {"cloud": "AzureChinaCloud"} if USE_CHINA_CLOUD else {}

# 交互式登录
credential = InteractiveBrowserCredential(authority=authority_host,tenant_id="16b3c013-d300-468d-ac64-7eda0820b6d3")
ml_client = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION_ID,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=WORKSPACE_NAME,
    **ml_client_kwargs,
)

print(f"Connected to workspace: {ml_client.workspace_name}")
workspace = ml_client.workspaces.get(ml_client.workspace_name)
print(f"Location: {workspace.location}")

Connected to workspace: aml-hu-east-west-us2
Location: westus2


## 2.2 准备目录结构

In [ ]:
import pathlib

workspace_dir = pathlib.Path.cwd() / "workshop_qwen_vl"
src_dir = workspace_dir / "src"
env_dir = workspace_dir / "env"
datasets_dir = workspace_dir / "datasets"
config_dir = workspace_dir / "config"
outputs_dir = workspace_dir / "outputs"

for d in [workspace_dir, src_dir, env_dir, datasets_dir, config_dir, outputs_dir]:
    d.mkdir(parents=True, exist_ok=True)

print("workspace_dir:", workspace_dir)
print("src_dir:", src_dir)
print("datasets_dir:", datasets_dir)
print("config_dir:", config_dir)
print("outputs_dir:", outputs_dir)

workspace_dir: /Users/chengbian/Documents/workspace/qwen_vl_lora/aml/workshop_qwen_vl
src_dir: /Users/chengbian/Documents/workspace/qwen_vl_lora/aml/workshop_qwen_vl/src
datasets_dir: /Users/chengbian/Documents/workspace/qwen_vl_lora/aml/workshop_qwen_vl/datasets
config_dir: /Users/chengbian/Documents/workspace/qwen_vl_lora/aml/workshop_qwen_vl/config
outputs_dir: /Users/chengbian/Documents/workspace/qwen_vl_lora/aml/workshop_qwen_vl/outputs


## 2.3 下载并处理 vqav2-small 数据集为 LLaMA-Factory JSON

数据集地址：https://huggingface.co/datasets/merve/vqav2-small

从本地 parquet 目录读取样本字节图像,导出到 `train_images/val_images`,并生成 `vqav2small_train.json` 与 `vqav2small_val.json`(messages/images 结构)。

In [4]:
import os, json, random
import pandas as pd
from PIL import Image
from io import BytesIO
from tqdm import tqdm

# 配置：若你已将 parquet 下载到本地目录，修改此路径
parquet_dir = os.path.join(datasets_dir.as_posix(), "merve--vqav2-small")
output_base_dir = os.path.join(datasets_dir.as_posix(), "vqav2small")
os.makedirs(output_base_dir, exist_ok=True)

max_samples = 1000  # 可改为 None 表示使用全部
val_ratio = 0.1

splits = {
    "train": {
        "img_dir": os.path.join(output_base_dir, "train_images"),
        "json_path": os.path.join(output_base_dir, "vqav2small_train.json"),
    },
    "val": {
        "img_dir": os.path.join(output_base_dir, "val_images"),
        "json_path": os.path.join(output_base_dir, "vqav2small_val.json"),
    },
}
for split in splits.values():
    os.makedirs(split["img_dir"], exist_ok=True)

print("读取所有 parquet 文件...")
all_rows = []
for file in sorted(os.listdir(parquet_dir)):
    if not file.endswith(".parquet"):
        continue
    df = pd.read_parquet(os.path.join(parquet_dir, file))
    for idx, row in df.iterrows():
        all_rows.append((file, idx, row))
print(f"共加载 {len(all_rows)} 条样本")

random.seed(42)
if max_samples and len(all_rows) > max_samples:
    all_rows = random.sample(all_rows, max_samples)

random.shuffle(all_rows)
split_index = int(len(all_rows) * (1 - val_ratio))
train_rows = all_rows[:split_index]
val_rows = all_rows[split_index:]
print(f"拆分结果：训练集 {len(train_rows)} 条，验证集 {len(val_rows)} 条")

def export_split(split_name, rows):
    cfg = splits[split_name]
    json_output = []
    print(f"开始导出 {split_name} 集")
    for file, idx, row in tqdm(rows):
        try:
            img_bytes = row["image"]["bytes"]
            img = Image.open(BytesIO(img_bytes)).convert("RGB")
        except Exception as e:
            print(f"图片解码失败: {file}_{idx}, 错误: {e}")
            continue
        img_name = f"{split_name}_{file}_{idx}.jpg"
        img_path = os.path.join(cfg["img_dir"], img_name)
        img.save(img_path)
        messages = [
            {"role": "user", "content": f"<image>{row['question']}"},
            {"role": "assistant", "content": row["multiple_choice_answer"]},
        ]
        json_output.append({"messages": messages, "images": [img_path]})
    with open(cfg["json_path"], "w", encoding="utf-8") as f:
        json.dump(json_output, f, ensure_ascii=False, indent=2)
    print(f"{split_name} 集导出完成，共 {len(json_output)} 条样本，图片保存在 {cfg['img_dir']}")

export_split("train", train_rows)
export_split("val", val_rows)
print("全部数据集导出完成。输出目录：", output_base_dir)

读取所有 parquet 文件...
共加载 21435 条样本
拆分结果：训练集 900 条，验证集 100 条
开始导出 train 集


100%|██████████| 900/900 [00:03<00:00, 299.22it/s]


train 集导出完成，共 900 条样本，图片保存在 /Users/chengbian/Documents/workspace/qwen_vl_lora/aml/workshop_qwen_vl/datasets/vqav2small/train_images
开始导出 val 集


100%|██████████| 100/100 [00:00<00:00, 469.74it/s]

val 集导出完成，共 100 条样本，图片保存在 /Users/chengbian/Documents/workspace/qwen_vl_lora/aml/workshop_qwen_vl/datasets/vqav2small/val_images
全部数据集导出完成。输出目录： /Users/chengbian/Documents/workspace/qwen_vl_lora/aml/workshop_qwen_vl/datasets/vqav2small


## 2.4 注册 Azure ML 数据资产

In [5]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
from datetime import datetime

# 将 vqav2small 目录注册为 Data 资产
vqav2_data_asset = Data(
    name="vqav2small-llamafactory",
    version=datetime.utcnow().strftime("%Y%m%d%H%M"),
    path=output_base_dir,   # 目录中包含 train_images/val_images 以及 json
    type=AssetTypes.URI_FOLDER,
    description="VQA vqav2-small preprocessed for LLaMA-Factory (messages/images)"
)
registered_vqav2 = ml_client.data.create_or_update(vqav2_data_asset)
print(f"Data asset created: {registered_vqav2.name}:{registered_vqav2.version}")
print("URI:", registered_vqav2.path)

/var/folders/ly/p_d2t4qx1pn8t41fwvpnykx00000gp/T/ipykernel_53321/3604369608.py:8: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  version=datetime.utcnow().strftime("%Y%m%d%H%M"),
Uploading vqav2small (50.14 MBs): 100%|██████████| 50137461/50137461 [00:14<00:00, 3489514.10it/s]




Data asset created: vqav2small-llamafactory:202510291437
URI: azureml://subscriptions/7a03e9b8-18d6-48e7-b186-0ec68da9e86f/resourcegroups/aml-rg/workspaces/aml-hu-east-west-us2/datastores/workspaceblobstore/paths/LocalUpload/6f1951706f548c8a1401400c9ea42a54ff5a9e1f0262a418343b51ff5b29aeba/vqav2small/


### 注册后可在azure的资产页面中查看
- ![查看资产](images/azure_ml_assets.png)

## 2.5 写入 LLaMA-Factory 训练配置 YAML(Qwen3-VL-4B LoRA)

生成 `qwen3vl_lora_sft.yaml`,设置 Qwen3-VL-4B-Instruct、LoRA 超参、batch/epochs、日志与输出目录。

In [6]:
yaml_path = (config_dir / "qwen3vl_lora_sft.yaml").as_posix()

yaml_content = f"""
### model
model_name_or_path: Qwen/Qwen3-VL-4B-Instruct
image_max_pixels: 262144
video_max_pixels: 16384
trust_remote_code: true

### method
stage: sft
do_train: true
finetuning_type: lora
lora_rank: 16
lora_target: all

### dataset
# 注意：此处使用自定义预处理后的 vqav2small_train.json；
# LLaMA-Factory 读取 JSON 路径时请用绝对路径或相对当前工作目录
# 如果需要，可改写为你的自定义数据集名称与路径
# 这里沿用 workshop 命名，具体解析逻辑以 llamafactory 的数据适配为准

# 兼容 workshop 的最小化配置；如需严格对齐 LF 官方数据注册，需在 data/dataset_info.json 中登记
# 此处只作为演示（实际训练时可在命令中通过 --dataset xxx 或传 data_path 参数）
dataset: vqav2small_train
template: qwen3_vl
cutoff_len: 2048
max_samples: 1000
overwrite_cache: true
preprocessing_num_workers: 16
dataloader_num_workers: 0

### output
output_dir: {outputs_dir.as_posix()}/saves/qwen3vl-4b/lora/sft
logging_steps: 10
save_steps: 200
plot_loss: true
overwrite_output_dir: true
save_only_model: false
report_to: none

### train
per_device_train_batch_size: 2
gradient_accumulation_steps: 8
learning_rate: 1.0e-4
num_train_epochs: 1.0
lr_scheduler_type: cosine
warmup_ratio: 0.1
bf16: true
ddp_timeout: 180000000
resume_from_checkpoint: null
"""

with open(yaml_path, "w", encoding="utf-8") as f:
    f.write(yaml_content)

print("写入配置:", yaml_path)
print("预期输出目录:", (outputs_dir / "saves/qwen3vl-4b/lora/sft").as_posix())

写入配置: /Users/chengbian/Documents/workspace/qwen_vl_lora/aml/workshop_qwen_vl/config/qwen3vl_lora_sft.yaml
预期输出目录: /Users/chengbian/Documents/workspace/qwen_vl_lora/aml/workshop_qwen_vl/outputs/saves/qwen3vl-4b/lora/sft


## 2.6 注册/创建 Azure ML 环境

#### 🎯 镜像选择建议

**当前配置（推荐）：HuggingFace NLP GPU 专用镜像**
```python
image="mcr.microsoft.com/azureml/curated/acft-hf-nlp-gpu:latest"
```
✅ **优点：**
- 专门为 HuggingFace Transformers 优化
- 预装 CUDA 12.x，完美支持最新的 bitsandbytes
- 针对 A100 GPU 优化
- 包含 NLP 和多模态任务的常用库

**其他可用选项：**

| 镜像 | CUDA | 适用场景 | 推荐度 |
|------|------|---------|--------|
| `acft-hf-nlp-gpu:latest` | 12.x | HuggingFace + LoRA 微调 | ⭐⭐⭐⭐⭐ |
| `pytorch-2.0-cuda11.7-cudnn8-ubuntu22.04:latest` | 11.7 | 通用 PyTorch 训练 | ⭐⭐⭐ |
| `openmpi4.1.0-cuda11.8-cudnn8-ubuntu22.04:latest` | 11.8 | 分布式训练 | ⭐⭐⭐ |

In [13]:
from azure.ai.ml.entities import Environment

aml_env = Environment(
    name="qwen3vl-lora-lf",
    description="Curated PyTorch image; pip installs LLaMA-Factory during job",
    image="mcr.microsoft.com/azureml/curated/acft-hf-nlp-gpu:latest",
)
registered_env = ml_client.environments.create_or_update(aml_env)
print(f"Environment registered: {registered_env.name}:{registered_env.version}")

Environment registered: qwen3vl-lora-lf:2


### 完成操作后可以在环境页面进行查看

- ![查看环境](images/azure_ml_environment.png)

## 2.7 创建并提交 LLaMA-Factory 训练命令作业(AML)

### 计算资源准备
1. 推荐使用带 A100 80GB 的 GPU（如 `Standard_NC24ads_A100_v4`）。
2. 若尚未创建计算集群，可在 Azure ML Studio → 计算 → 计算集群 创建，或使用 Azure CLI：
   ```bash
   az ml compute create --name gpu-a100-cluster --type amlcompute \
       --resource-group <RG> --workspace-name <WS> \
       --min-instances 0 --max-instances 2 --size Standard_NC24ads_A100_v4
   ```
3. 训练前确认集群处于“空闲”或“已分配”状态，确保配额足够。

In [9]:
# 检查现有计算资源并在需要时创建 GPU 集群
from azure.ai.ml.entities import AmlCompute

# 列出所有现有的计算资源
print("现有计算资源:")
for compute in ml_client.compute.list():
    print(f"  - {compute.name} (类型: {compute.type}, 大小: {getattr(compute, 'size', 'N/A')}, 状态: {compute.provisioning_state})")

# 定义计算集群名称
COMPUTE_NAME = "qwen-fine-tune-H100"

# 检查计算集群是否存在，如果不存在则创建
try:
    compute_target = ml_client.compute.get(COMPUTE_NAME)
    print(f"\n计算集群 '{COMPUTE_NAME}' 已存在，状态: {compute_target.provisioning_state}")
except Exception as e:
    print(f"\n计算集群 '{COMPUTE_NAME}' 不存在，正在创建...")
    
    # 创建计算集群配置
    # 注意：Standard_NC24ads_A100_v4 在某些区域可能不可用
    # 可替换为其他 GPU SKU，如 Standard_NC6s_v3, Standard_NC12s_v3 等
    compute_config = AmlCompute(
        name=COMPUTE_NAME,
        type="amlcompute",
        size="Standard_NC80adis_H100_v5",  # A100 80GB GPU
        min_instances=0,
        max_instances=1,
        idle_time_before_scale_down=300,  # 5分钟后自动缩减
        tier="dedicated",
    )
    
    try:
        compute_target = ml_client.compute.begin_create_or_update(compute_config).result()
        print(f"计算集群 '{COMPUTE_NAME}' 创建成功！")
    except Exception as create_error:
        print(f"创建失败: {create_error}")
        print("\n可能的原因:")
        print("1. 所选 GPU SKU 在当前区域不可用")
        print("2. GPU 配额不足")
        print("\n建议:")
        print("- 在 Azure Portal 中检查可用的 VM SKU")
        print("- 请求增加 GPU 配额")
        print("- 或使用现有的计算资源（如果上面列出了可用的计算）")
        raise

现有计算资源:
  - A100-swedCentral (类型: virtualmachine, 大小: N/A, 状态: Succeeded)
  - A100-centre-us (类型: virtualmachine, 大小: N/A, 状态: Succeeded)
  - multi-model-batch-vm (类型: computeinstance, 大小: Standard_D96ads_v5, 状态: Succeeded)
  - o1-performance-test-vm (类型: computeinstance, 大小: Standard_E4ds_v4, 状态: Succeeded)
  - slm-fine-tune-lab (类型: computeinstance, 大小: Standard_E4ds_v4, 状态: Succeeded)
  - gpu-cluster-a100 (类型: amlcompute, 大小: Standard_NC24ads_A100_v4, 状态: Succeeded)
  - agent-demo-ws (类型: computeinstance, 大小: Standard_D32d_v4, 状态: Succeeded)
  - notebook-llm-solu-vm (类型: computeinstance, 大小: Standard_D48a_v4, 状态: Succeeded)
  - gpu-phi4-predict-out (类型: computeinstance, 大小: Standard_NC8as_T4_v3, 状态: Succeeded)
  - Standard-NV12s-v3-m60 (类型: computeinstance, 大小: Standard_NV12s_v3, 状态: Succeeded)
  - multi-gpus-trainer (类型: computeinstance, 大小: Standard_NC64as_T4_v3, 状态: Succeeded)
  - a100-west-1 (类型: amlcompute, 大小: Standard_NC24ads_A100_v4, 状态: Succeeded)
  - qwen-fine-tune-A100 (类

### 完成计算资源准备后， 提交训练命令
命令会在计算节点上:
1) 克隆 LLaMA-Factory 并安装;2) 设置 HF 镜像与加速;3) 读取挂载的数据目录;4) 调用 `llamafactory-cli train` 使用上文 YAML 配置。


In [14]:
from azure.ai.ml import Input, Output, command
from azure.ai.ml.constants import AssetTypes

# 计算与实验配置（请根据你的工作区实际值修改）
EXPERIMENT_NAME = "qwen3vl-lora-lf"

# 使用上面注册的 Data 资产作为输入
data_input = Input(type=AssetTypes.URI_FOLDER, path=registered_vqav2.id)

# 训练命令：在节点上安装 LLaMA-Factory 并运行训练
# 注意：${{inputs.data}} 为 Azure ML 在运行时注入的挂载路径
train_cmd = " && ".join([
    "set -e",
    "echo \"Python:\" $(python --version)",
    "echo \"Pip:\" $(pip --version)",
    "git clone https://github.com/hiyouga/LLaMA-Factory.git",
    "cd LLaMA-Factory",
    "export HF_HUB_ENABLE_HF_TRANSFER=1",
    "if [ \"$AZURE_CLOUD_NAME\" = \"AzureChinaCloud\" ]; then export HF_ENDPOINT=https://hf-mirror.com; fi",
    "pip install -e \".[torch,metrics]\" --no-build-isolation",
    f"llamafactory-cli train {yaml_path}",
])

job = command(
    code=str(workspace_dir),
    command=train_cmd,
    inputs={"data": data_input},
    environment=registered_env,
    compute=COMPUTE_NAME,
    experiment_name=EXPERIMENT_NAME,
    display_name="qwen3vl-lora-train-lf",
    outputs={
        "trained": Output(type=AssetTypes.URI_FOLDER)  # 将默认写入作业输出目录
    },
    description="Qwen3-VL-4B LoRA training via LLaMA-Factory"
)

# 以挂载方式访问数据（提升 I/O 速度）
job.inputs["data"].mode = "mount"
submitted_job = ml_client.jobs.create_or_update(job)
print("Job submitted:", submitted_job.name)

pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored


Job submitted: cool_head_rrkrh8znqf


## 2.8 监控训练日志(AML)

In [ ]:
ml_client.jobs.stream(submitted_job.name)

## 2.9 下载训练输出并展示损失曲线

In [ ]:
import pathlib, tempfile
from PIL import Image
from IPython.display import display

# 下载作业产物到本地临时目录
local_download = pathlib.Path(tempfile.mkdtemp(prefix="aml-train-"))
ml_client.jobs.download(submitted_job.name, download_path=local_download.as_posix())
print("Downloaded to:", local_download)

# 搜索 training_loss.png
loss_png = None
for p in local_download.rglob("training_loss.png"):
    loss_png = p
    break

if loss_png and loss_png.exists():
    print("Found:", loss_png)
    display(Image.open(loss_png).convert("RGB"))
else:
    print("未找到 training_loss.png。请检查作业输出目录结构。")

## 2.10 合并 LoRA 适配器为完整模型(AML 导出作业)

使用第二个命令作业在计算节点上运行 `llamafactory-cli export`,将前一作业的 LoRA 适配器与基础模型合并。

In [ ]:
from azure.ai.ml import Input

# 将上一个作业的默认 outputs 作为输入传入
prev_outputs_uri = f"azureml://jobs/{submitted_job.name}/outputs"
prev_input = Input(type=AssetTypes.URI_FOLDER, path=prev_outputs_uri)

merge_cmd = " && ".join([
    "set -e",
    "git clone https://github.com/hiyouga/LLaMA-Factory.git",
    "cd LLaMA-Factory",
    "pip install -e \".[torch,metrics]\" --no-build-isolation",
    # 选择最近的 checkpoint 目录
    "ADAPTER=$(ls -d ${inputs.prev}/saves/qwen3vl-4b/lora/sft/checkpoint-* | sort | tail -n 1)",
    "echo Using adapter: $ADAPTER",
    # 生成合并配置文件
    "cat > merge_lora.yaml <<'EOF'\n"
    "### model\n"
    "model_name_or_path: Qwen/Qwen3-VL-4B-Instruct\n"
    "adapter_name_or_path: $ADAPTER\n"
    "template: qwen3_vl\n"
    "trust_remote_code: true\n\n"
    "### export\n"
    "export_dir: ./outputs/merged\n"
    "export_size: 5\n"
    "export_device: cpu\n"
    "export_legacy_format: false\n"
    "EOF",
    # 执行导出
    "llamafactory-cli export merge_lora.yaml",
])

merge_job = command(
    code=str(workspace_dir),
    command=merge_cmd,
    inputs={"prev": prev_input},
    environment=registered_env,
    compute=COMPUTE_NAME,
    experiment_name=EXPERIMENT_NAME,
    display_name="qwen3vl-merge-lora",
    outputs={"merged": Output(type=AssetTypes.URI_FOLDER)},
    description="Merge LoRA adapter into full model"
)

merge_job.inputs["prev"].mode = "download"  # 读取上次作业产物
submitted_merge = ml_client.jobs.create_or_update(merge_job)
print("Merge job submitted:", submitted_merge.name)

In [ ]:
# 监控合并作业
ml_client.jobs.stream(submitted_merge.name)

## 2.11 注册 LoRA 适配器/合并模型为 Azure ML 模型资产

In [ ]:
from azure.ai.ml.entities import Model

BASE_MODEL = "Qwen/Qwen3-VL-4B-Instruct"

# 适配器模型（注册训练作业的 outputs 目录）
adapter_uri = f"azureml://jobs/{submitted_job.name}/outputs"
adapter_model = Model(
    name="qwen3vl-lora-adapter-lf",
    path=adapter_uri,
    type=AssetTypes.URI_FOLDER,
    description="LoRA adapter trained via LLaMA-Factory on AML",
    tags={"base_model": BASE_MODEL, "task": "sft", "tool": "llamafactory"}
)
reg_adapter = ml_client.models.create_or_update(adapter_model)
print("Adapter model:", f"{reg_adapter.name}:{reg_adapter.version}")

# 合并后完整模型（注册导出作业的 merged 输出）
merged_uri = f"azureml://jobs/{submitted_merge.name}/outputs/merged"
full_model = Model(
    name="qwen3vl-merged-lf",
    path=merged_uri,
    type=AssetTypes.URI_FOLDER,
    description="Full model merged from base + LoRA via LLaMA-Factory export",
    tags={"base_model": BASE_MODEL, "merged": "true", "tool": "llamafactory"}
)
reg_full = ml_client.models.create_or_update(full_model)
print("Merged model:", f"{reg_full.name}:{reg_full.version}")